In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Workflow
1. Import datasets
1. EDA
1. Preprocessing
1. Modeling
1. Submit

# Import datasets

In [ ]:
train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
pd.options.display.max_columns = None

In [ ]:
train.head()

# EDA

In [ ]:
train.info()

In [ ]:
train.isnull().sum(), test.isnull().sum()

# Preprocessing

In [ ]:
# Impute: winPlacePerc
train['winPlacePerc'] = train['winPlacePerc'].fillna(train['winPlacePerc'].mean())
# test['winPlacePerc'] => 'winPlacePerc'はターゲットなのでなし

In [ ]:
# Index
train = train.set_index('Id')
test = test.set_index('Id')

In [ ]:
# Drop: groupId, matchId
cols_to_drop = ['groupId', 'matchId']
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [ ]:
# Encode: matchType
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train['matchType'] = label_encoder.fit_transform(train['matchType'])
test['matchType'] = label_encoder.transform(test['matchType'])

In [ ]:
train.head()

In [ ]:
test.head()

# Modeling

In [ ]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X = train.drop('winPlacePerc', axis=1)
y = train['winPlacePerc']
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
# Model
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor()

In [ ]:
# Fit
lgbm.fit(X_train, y_train)

In [ ]:
# Predict, Evaluate
from sklearn.metrics import mean_absolute_error
mean_absolute_error(lgbm.predict(X_valid), y_valid)

# Submit

In [ ]:
# Predict
predictions = lgbm.predict(test)
predictions

In [ ]:
# Submit
submissions = pd.DataFrame({
    'Id': test.index,
    'winPlacePerc': predictions
})
submissions.to_csv('submission.csv', index=False)